In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping


Bad key "text.kerning_factor" on line 4 in
D:\conda\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
#reading the preprocessed dataset
x=pd.read_csv('x.csv')
y=pd.read_csv('y.csv')

# Data Normalization


In [3]:
#splitting data into test train
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20)
X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size=0.20) # 0.25 x 0.8 = 0.2

In [4]:

#Min-max Data normalization
min_max_scaler = preprocessing.MinMaxScaler()
X_train_scale = min_max_scaler.fit_transform(X_train)
X_val_scale = min_max_scaler.fit_transform(X_val)
X_test_scale = min_max_scaler.fit_transform(X_test)

# Modeling 


In [6]:

#from tensorflow.keras.optimizers import Lookahead

n_hidden1 = 135
n_hidden2 = 128
n_hidden3 = 64
n_hidden4 = 32

# create model
model = Sequential()

model.add(Dense(n_hidden1, input_dim=X_train_scale.shape[1], activation='relu'))
#model.add(Dense(256, activation='relu'))

model.add(Dense(n_hidden2, activation='relu'))
model.add(Dense(n_hidden3, activation='relu'))
model.add(Dense(n_hidden4, activation='relu'))

model.add(Dropout(0.5))

#model.add(Dense(256, activation='relu'))
model.add(Dense(n_hidden2, activation='relu'))
model.add(Dense(n_hidden3, activation='relu'))
model.add(Dense(n_hidden4, activation='relu'))

model.add(Dropout(0.5))
#model.add(Dense(256, activation='relu'))
model.add(Dense(n_hidden2, activation='relu'))
model.add(Dense(n_hidden3, activation='relu'))
model.add(Dense(n_hidden4, activation='relu'))

model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))


In [10]:
callbacks = [EarlyStopping(monitor='val_loss', patience=3)]

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model


model.fit(X_train_scale, np.array(y_train), epochs=50, batch_size=20, callbacks=callbacks,validation_data=(X_val_scale, np.array(y_val)))

Train on 361837 samples, validate on 90460 samples
Epoch 1/50
361837/361837 [==============================] - 63s 174us/sample - loss: 0.3684 - accuracy: 0.8353 - val_loss: 0.2836 - val_accuracy: 0.8930
Epoch 2/50
361837/361837 [==============================] - 57s 157us/sample - loss: 0.2551 - accuracy: 0.8974 - val_loss: 0.2428 - val_accuracy: 0.9068
Epoch 3/50
361837/361837 [==============================] - 67s 184us/sample - loss: 0.2286 - accuracy: 0.9111 - val_loss: 0.2171 - val_accuracy: 0.9227
Epoch 4/50
361837/361837 [==============================] - 54s 149us/sample - loss: 0.2141 - accuracy: 0.9188 - val_loss: 0.5524 - val_accuracy: 0.7027
Epoch 5/50
361837/361837 [==============================] - 86s 239us/sample - loss: 0.2038 - accuracy: 0.9237 - val_loss: 0.2126 - val_accuracy: 0.9249
Epoch 6/50
361837/361837 [==============================] - 87s 241us/sample - loss: 0.2003 - accuracy: 0.9255 - val_loss: 0.2769 - val_accuracy: 0.8878
Epoch 7/50
361837/361837 [=====

In [12]:
model.save('NN_final')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: NN_final\assets


# Evaluation 

- Precision, Recall

In [11]:
#imblance classes 
y_pred = model.predict(X_test_scale)
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred.round()))
#print('Accuracy of rf classifier on test set: {:.2f}'.format(model.score(X_test_scale, y_test)))
from sklearn.metrics import roc_auc_score
print("AUC of rf classifier on test set", roc_auc_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.74      0.82     56529
           1       0.78      0.95      0.86     56546

    accuracy                           0.84    113075
   macro avg       0.86      0.84      0.84    113075
weighted avg       0.86      0.84      0.84    113075

AUC of rf classifier on test set 0.9668039872777481
